In [2]:
import pandas as pd
from lxml import html
import requests
import datetime

In [3]:
nicknames = {"Hawks":"ATL","Celtics":"BOS","Nets":"BRK","Hornets":"CHO","Bulls":"CHI","Cavaliers":"CLE",
             "Mavericks":"DAL","Nuggets":"DEN","Pistons":"DET","Warriors":"GSW","Rockets":"HOU","Pacers":"IND",
             "Clippers":"LAC","Lakers":"LAL","Grizzlies":"MEM","Heat":"MIA","Bucks":"MIL","Timberwolves":"MIN",
             "Pelicans":"NOP","Knicks":"NYK","Thunder":"OKC","Magic":"ORL","76ers":"PHI","Suns":"PHO",
             "Blazers":"POR","Kings":"SAC","Spurs":"SAS","Raptors":"TOR","Jazz":"UTA","Wizards":"WAS"}
             
teams = list(nicknames.values())

In [ ]:
def get_player_info():
    """
    Get HTML tables containing player info for each team roster on basketball-reference.com 
    """
    # Add player data from HTML tables to a dictionary with team abbreviations as the keys
    rosters = {}
    for team in teams:
        url = f"https://www.basketball-reference.com/teams/{team}/2022.html#all_roster"
        roster = pd.read_html(url)[0]
        roster.drop(columns=["No.","Unnamed: 6","College"],inplace=True)

        # Change birth date to each player's current age, and change height to inches for easier data analysis
        ages = []
        inches = []
        for info in zip(roster["Birth Date"],roster["Ht"]):
            age = int((datetime.datetime.now()-datetime.datetime.strptime(info[0],'%B %d, %Y')).days/365.25)
            ages.append(age)

            height = info[1].split("-")
            ht_inches = int(height[0])*12+int(height[1])
            inches.append(ht_inches)


        roster["Birth Date"] = pd.Series(ages)
        roster.rename(columns={"Birth Date": "Age"}, inplace=True)
        roster["Ht"] = pd.Series(inches)
        roster["Exp"] = roster["Exp"].replace({"R":"0"}).astype("int64") # Change rookie experience values to zero for converting series to int
        roster["Player"] = [name.rstrip(" (TW)") for name in roster["Player"]] # Remove unnecessary indicator for two-way players

        # Get each rostered player's site ID via html request
        response = requests.get(url)
        tree = html.fromstring(response.content)
        roster["ID"] = [tree.xpath(f'//*[@id="roster"]/tbody/tr[{i}]/td[1]/a')[0].values()[0].split("/")[3].split(".")[0] for i in range(1,len(roster)+1)]
        rosters[team] = roster
        
    # Condense all team dictionaries into a single dataframe
    for i,tm in enumerate(rosters):
        rosters[tm]["Team"] = [tm for _ in range(len(rosters[tm]))]
        if i==0:
            players = rosters[tm]
        else:
            players = players.append(rosters[tm])
    
    # re-arrange columns and drop all suffixes from player names
    players = players[["Player","Team","Pos","Ht","Wt","Age","Exp","ID"]].reset_index(drop=True)
    players["Player"] = [name.replace(" Jr.","").replace(" Sr.","").replace(" II","").replace(" III","").replace(" IV","") for name in players["Player"]]
    return players

players = get_player_info()

In [5]:
def get_player_logs():
    """
    Get HTML tables containing player logs from basketball-reference.com
    """
    # Organize each player's individual game logs into a dictionary with player's name as the key
    player_logs = {}
    for i in players.index:
        try: # Player logs for some players with limited playing time omit some standard columns, which breaks the below code 
            player, id = players["Player"][i], players["ID"][i]
            url = f'https://www.basketball-reference.com/players/{id[0]}/{id}/gamelog/2022'
            log = pd.read_html(url,attrs={"id":"pgl_basic"})[0].drop(columns=["Rk","G","Age","FG%","FT%"])
            # Drop, rename, and create columns
            if "3P%" in log.columns:
                log = log.drop(columns="3P%")
            for i in log.index:
                if (i % 20 == 0 and i != 0) or not str(log["PTS"][i]).isnumeric():
                    log.drop(i,inplace=True)
            log["Unnamed: 5"] = ["Road" if i=="@" else "Home" for i in log["Unnamed: 5"]]
            log.rename(columns={"Unnamed: 5": "Court"}, inplace=True)
            log["W/L"] = [i.split(" ")[0] for i in log["Unnamed: 7"]]
            log["Spread"] = [i.split(" ")[1] for i in log["Unnamed: 7"]]
            log["Spread"] = [int(i.split("+")[1].rstrip(")")) if "+" in i else int(i.split("(")[1].rstrip(")")) for i in log["Spread"]]
            log["+/-"] = log["+/-"].fillna(0).astype("int64")
            log.drop(columns=("Unnamed: 7"),inplace=True)
            # Change datatypes of numeric columns to ints and convert decimal columns to floats
            for i in log:
                try:
                    log[i] = log[i].astype("int64")
                except:
                    pass
            log["MP"] = [round(int(i.split(":")[0]) + int(i.split(":")[1])/60,1) for i in log["MP"]] # Convert minutes played to a decimal
            log["GmSc"] = log["GmSc"].astype("float64")
            stats = ["Date","Tm","Opp","Court","W/L","Spread","+/-","GS","MP","GmSc","PTS","FG",
                     "FGA","3P","3PA","FT","FTA","ORB","DRB","TRB","AST","BLK","STL","TOV","PF"]
            # Fill empty stats with zeroes
            for stat in stats:
                if stat not in log.columns:
                    log[stat] = [0 for i in range(len(log))]
            player_logs[player] = log[stats].reset_index(drop=True)
        except:
            pass
    return player_logs
    
player_logs = get_player_logs()

In [6]:
def get_games():
    games = {}
    for player_log in [(name,player_logs[name][player_logs[name].columns]) for name in player_logs]:
        name = player_log[0]
        log = player_log[1]
        log["Player"] = name
        for idx in log.index:
            tms = sorted([log["Tm"][idx],log["Opp"][idx]])
            MU = f'{tms[0]}-{tms[1]}'
            date = log["Date"][idx]
            if MU not in games:
                games[MU] = {}
            if date not in games[MU]:
                games[MU][date] = pd.DataFrame()
            if date not in games:
                games[date] = {}
            if MU not in games[date]:
                games[date][MU] = pd.DataFrame()
            games[MU][date] = pd.concat([games[MU][date],log[idx:idx+1]]).reset_index(drop=True)
            games[date][MU] = pd.concat([games[date][MU],log[idx:idx+1]]).reset_index(drop=True)
    return games
    
games = get_games()

In [7]:
def team_logs(team,opponent="ANY"):
    if team in teams:
        MU_info = []
        team_games = []
        for MU in games:
            if opponent in teams:
                if team in MU and opponent in MU:
                    for date in list(games[MU]):
                        MU_info.append((date,MU))
            else:
                if team in MU:
                    for date in list(games[MU]):
                        MU_info.append((date,MU))
        for info in sorted(MU_info):
            team_games.append(games[info[0]][info[1]])
            
        cols = ["Player","Team","Pos","Ht","Wt","GS","MP","FG","FGA","3P",
                "3PA","FT","FTA","PTS","TRB","AST","BLK","STL","TOV","PF"]
        for i,v in enumerate(team_games):
            team_games[i] = pd.merge(players[cols[:5]],v,how="inner")[cols].sort_values(["Team","GS","PTS"],ascending=False,ignore_index=True)

        return team_games
    raise NameError("You entered an invalid team abbrevation")

In [8]:
def get_player_offense(filter="totals",team="ALL",position="All",minutes=0):
    aggregates = {}
    for player in player_logs:
        if player_logs[player]["MP"].sum()>minutes:
            aggregates[player] = {}
            # aggregates[player]["GP"] = len(player_logs[player])
            for stat in player_logs[player]:
                if player_logs[player][stat].dtype == "int64" or player_logs[player][stat].dtype == "float64":
                    if filter == "totals":
                        aggregates[player][stat] = player_logs[player][stat].sum()
                    elif filter == "means":
                        aggregates[player][stat] = round(player_logs[player][stat].mean(),1)
                    elif filter == "per_36":
                        aggregates[player][stat] = round(player_logs[player][stat].sum()/player_logs[player]["MP"].sum()*36,1)

    leaderboard = pd.DataFrame(aggregates).transpose().dropna()
    leaderboard = pd.merge(players[players.columns[:6]],leaderboard,how="inner",left_on="Player",right_on=leaderboard.index)

    cols = (list(leaderboard.columns[:6])+list(leaderboard.columns[9:25]))
    cols.remove("ORB"), cols.remove("DRB")

    if filter=="totals":
        for stat in cols[-12:]:
            leaderboard[stat] = leaderboard[stat].astype("int64")
    if team in teams:
        leaderboard = leaderboard[leaderboard["Team"]==team]
    if position in players["Pos"].unique():
        leaderboard = leaderboard[leaderboard["Pos"]==position]

    return leaderboard[cols].sort_values(by="GmSc",ascending=False)

In [9]:
def get_team_offense(filter="means",position="ALL",minutes=0,min_ht=min(players["Ht"]),max_ht=max(players["Ht"])):
    totals = get_player_offense("totals")
    cols = [col for col in totals if totals[col].dtype=="int64"]
    team_totals = {}
    for tm in teams:
        team = totals[totals["Team"]==tm]
        if position in players["Pos"].unique():
            team = team[team["Pos"]==position]
        team = team[(team["Ht"]>=min_ht) & (team["Ht"]<=max_ht)]
        team_totals[tm] = [team[col].sum() for col in cols]

    df = pd.DataFrame(team_totals).transpose()
    for stat,col in zip(df,cols):
        df.rename(columns={stat:col},inplace=True)
    if filter=="totals":
        return df.sort_values(by="PTS",ascending=False)

    if filter=="means":
        for stat in df:
            df[stat] = df[stat].astype("float64")
        for team in teams:
            GP = 0
            MUs = [list(games[i].keys()) for i in games if team in i]
            for i in MUs:
                for j in i:
                    GP += 1
            for stat in df.columns[:3]:
                df.loc[team][stat] = round(players[players["Team"]==team][stat].mean(),1)
            for stat in df.columns[3:]:
                df.loc[team][stat] = round(df.loc[team][stat]/GP,1)
        return df.sort_values(by="PTS",ascending=False)
    raise NameError("The filter parameter must be either 'totals' or 'means")

In [10]:
def get_team_defense(filter="totals",position="ALL",roster="FULL",min_ht=min(players["Ht"]),max_ht=max(players["Ht"])):
    if filter=="totals" or filter=="means" or filter=="per_36":
        team_totals = {}
        for team in teams:
            opp_logs = pd.DataFrame()
            logs = team_logs(team)
            for log in logs:
                opp_stats = log[~log["Player"].isin(players[players["Team"]==team]["Player"])]
                if position in players["Pos"].unique():
                    opp_stats = opp_stats[opp_stats["Pos"]==position]
                if type(roster)==int:
                    opp_stats = opp_stats[opp_stats["GS"]==roster]
                opp_stats = opp_stats[(opp_stats["Ht"]>=min_ht) & (opp_stats["Ht"]<=max_ht)]
                opp_logs = pd.concat([opp_stats,opp_logs])
            cols_sums = [("GP",len(logs))]+[(stat,opp_logs[stat].sum()) for stat in opp_logs.columns[6:] if opp_logs[stat].dtype=="int64" or opp_logs[stat].dtype=="float64"]
            team_totals[team] = [sums[1] for sums in cols_sums]
        cols = [cols[0] for cols in cols_sums]
        df = pd.DataFrame(team_totals).transpose()
        for name,num in zip(cols,df.columns):
            df.rename({num:name},axis=1,inplace=True)
            if filter=="means" and name!="GP":
                df[name] = round(df[name]/df["GP"],1)
            elif filter=="per_36" and name!="GP" and name!="MP":
                df[name] = round(df[name]/df["MP"]*36,1)
            else:
                df[name] = df[name].astype("int64")
        if filter=="means" or filter=="per_36":
            df["FG%"] = round(df["FG"]/df["FGA"],3)
            df["3P%"] = round(df["3P"]/df["3PA"],3)
        return df.sort_values(by="PTS")
    raise NameError("Filter parameter only accepts three keywords: 'totals,' 'means,' or 'per_36'")


team_def_means = get_team_defense("means")
team_def_C  = get_team_defense("per_36","C")
team_def_PF = get_team_defense("per_36","PF")
team_def_SF = get_team_defense("per_36","SF")
team_def_SG = get_team_defense("per_36","SG")
team_def_PG = get_team_defense("per_36","PG")
# team_def_starters = get_team_defense("per_36",roster=1)
# team_def_bench = get_team_defense("per_36",roster=0)

In [11]:
def get_injury_report(team="ALL"):
    df = pd.read_html("https://www.basketball-reference.com/friv/injuries.fcgi")[0]

    designation = []
    for i in df["Description"]:
        if "out" in i.lower():
            designation.append("Out")
        elif "doubtful" in i.lower():
            designation.append("Doubtful")
        elif "probable" in i.lower():
            designation.append("Probable")
        else:
            designation.append("Questionable")

    df["Designation"] = designation
    df["Update"] = [(datetime.datetime.now()-datetime.datetime.strptime(i,"%a, %b %d, %Y")).days for i in df["Update"]]
    df["Team"] = [nicknames[i.split()[-1]] for i in df["Team"]]
    df["Type"] = [i.split("(")[1].split(")")[0] for i in df["Description"]]
    df["Surgery"] = ["surgery" in i.lower() for i in df["Description"]]
    
    df = df[["Player","Team","Designation","Update","Type","Surgery","Description"]].sort_values(by="Update")
    if team in teams:
        return df[df["Team"]==team]
    return df


def injury_filter(team,filter="means",minutes=0):
    df = get_player_offense(filter,team=team,minutes=minutes)
    return df[~df["Player"].isin(get_injury_report()["Player"])]

injury_report = get_injury_report()

In [12]:
def get_matchups(date_=datetime.date.today()):
    month_ = date_.strftime("%B").lower()
    url = f"https://www.basketball-reference.com/leagues/NBA_2022_games-{month_}.html"
    df = pd.read_html(url)[0].rename(columns={"Visitor/Neutral":"Road","Home/Neutral":"Home"})
    df = df.iloc[:,:5].drop(columns="PTS")
    df["Date"] = [datetime.datetime.strptime(i,"%a, %b %d, %Y").date() for i in df["Date"]]
    df["Road"] = [nicknames[i.split()[-1]] for i in df["Road"]]
    df["Home"] = [nicknames[i.split()[-1]] for i in df["Home"]]
    df["MU"] = [f"{sorted(i)[0]}-{sorted(i)[1]}" for i in df[["Road","Home"]].values]
    df = df[df["Date"]==date_]
    return df

games_today = get_matchups()

In [13]:
def get_dashboard(team,category):
    category == category.capitalize()
    if team not in teams:
        raise NameError("Invalid team selection")
    if category not in ["History","Injuries","Rosters","Overall"]+list(players["Pos"].unique()):
        raise NameError("Invalid category selection")

    if team in list(games_today["Home"])+list(games_today["Road"]):
        dfs = {"PG":team_def_PG,"SG":team_def_SG,"SF":team_def_SF,"PF":team_def_PF,"C":team_def_C,"Overall":team_def_means}
        cols = ["Player","Tm","Court","GS","MP","PTS","AST","TRB","FG","FGA",
                "3P","3PA","FT","FTA","BLK","STL","TOV","PF","+/-","GmSc"]

        for MU in games_today["MU"]:
            if team in MU:
                tm1,tm2 = MU.split("-")[0], MU.split("-")[1]
                if category=="History":
                    return team_logs(tm1,tm2)
                if category=="Injuries":
                    return injury_report[(injury_report["Team"]==tm1) | (injury_report["Team"]==tm2)].sort_values("Team")
                if category=="Rosters":
                    return injury_filter(tm1,minutes=200).append(injury_filter(tm2,minutes=200))
                if category in players["Pos"].unique() or category=="Overall":
                    df = dfs[category]
                    aggregates = df.agg(["mean","std"])
                    for i in aggregates:
                        df[i] = round((df[i]-aggregates[i]["mean"])/aggregates[i]["std"],2)
                    return df[(df.index==tm1) | (df.index==tm2)]

    return f"{team} doesn't play today"

In [14]:
games_today

,Date,Start (ET),Road,Home,MU


In [43]:
# get_dashboard("ATL","Injuries")

,Player,Team,Designation,Update,Type,Surgery,Description
4,Timothé Luwawu-Cabarrot,ATL,Out,1,Health and safety protocols,False,Out (Health and safety protocols) - The Hawks ...
5,Lou Williams,ATL,Out,1,Health and safety protocols,False,Out (Health and safety protocols) - The Hawks ...
0,Clint Capela,ATL,Out,2,Health and Safety Protocols,False,Out (Health and Safety Protocols) - The Hawks ...
1,Danilo Gallinari,ATL,Out,2,Health and Safety Protocols,False,Out (Health and Safety Protocols) - The Hawks ...
6,Trae Young,ATL,Out,4,Health and safety protocols,False,Out (Health and safety protocols) - The Hawks ...
2,Solomon Hill,ATL,Out,15,Hamstring,False,Out For Season (Hamstring) - The Hawks announc...
3,De'Andre Hunter,ATL,Out,39,Wrist,True,Out (Wrist) - The Hawks announced that Hunter ...
132,Danny Green,PHI,Out,1,Health Protocols,False,Out (Health Protocols) - Green is listed as OU...
133,Furkan Korkmaz,PHI,Questionable,2,Illness,False,Day To Day (Illness) - Korkmaz did not play in...
131,Andre Drummond,PHI,Out,4,Health and safety protocols,False,Out (Health and safety protocols) - The 76ers ...


In [44]:
# injury_filter("ATL","per_36",minutes=0)

,Player,Team,Pos,Ht,Wt,Age,MP,GmSc,PTS,FG,FGA,3P,3PA,FT,FTA,TRB,AST,BLK,STL,TOV
16,Onyeka Okongwu,ATL,C,80,235,21,36.0,17.9,19.9,6.6,7.7,0.0,0.0,6.6,6.6,9.9,1.1,3.3,1.1,3.3
0,John Collins,ATL,PF,81,235,24,36.0,17.0,19.4,7.3,13.1,1.4,3.5,3.4,4.2,8.9,2.3,1.4,0.8,1.2
13,Jalen Johnson,ATL,PF,81,220,20,36.0,15.2,21.7,7.2,15.7,2.4,6.0,4.8,7.2,7.2,1.2,0.0,0.0,0.0
5,Cam Reddish,ATL,SF,80,218,22,36.0,12.3,18.7,6.2,15.1,2.8,7.0,3.4,3.8,4.1,1.6,0.4,1.6,1.8
14,Skylar Mays,ATL,SG,76,205,24,36.0,11.7,17.9,5.1,7.7,1.3,3.8,6.4,6.4,3.8,1.3,0.0,0.0,3.8
7,Gorgui Dieng,ATL,C,82,252,31,36.0,11.4,11.8,4.1,9.4,1.6,4.6,2.0,2.8,13.0,2.6,1.5,1.2,2.3
10,Bogdan Bogdanović,ATL,SG,78,220,29,36.0,10.6,14.6,5.6,12.8,2.7,7.1,0.7,1.0,4.5,3.1,0.2,0.9,1.1
3,Kevin Huerter,ATL,SG,79,190,23,36.0,9.6,13.7,5.4,11.8,2.3,6.0,0.6,0.7,4.3,3.5,0.3,0.6,1.7
4,Delon Wright,ATL,SG,77,185,29,36.0,9.6,7.3,2.6,6.6,0.8,2.2,1.3,1.6,6.6,5.2,0.4,1.6,1.7
15,Sharife Cooper,ATL,PG,73,180,20,36.0,-4.3,4.8,2.4,14.4,0.0,4.8,0.0,0.0,3.6,6.0,0.0,0.0,4.8


In [46]:
# injury_filter("PHI","per_36",minutes=100)

,Player,Team,Pos,Ht,Wt,Age,MP,GmSc,PTS,FG,FGA,3P,3PA,FT,FTA,TRB,AST,BLK,STL,TOV
303,Joel Embiid,PHI,C,84,280,27,36.0,22.1,27.0,8.5,18.7,1.3,3.8,8.8,10.9,11.7,4.7,1.5,1.2,2.9
306,Charles Bassey,PHI,PF,83,235,21,36.0,15.0,14.3,6.2,9.3,0.0,0.9,1.9,2.8,13.1,0.9,3.7,0.6,1.6
302,Tobias Harris,PHI,PF,80,226,29,36.0,14.6,19.5,7.4,16.1,1.4,4.3,3.3,3.9,8.0,3.5,0.6,0.4,1.7
295,Tyrese Maxey,PHI,PG,74,200,21,36.0,13.9,17.7,6.7,13.9,1.2,3.3,3.1,3.6,3.6,4.9,0.6,0.7,1.4
296,Seth Curry,PHI,SG,74,185,31,36.0,12.6,17.2,6.5,12.3,2.2,5.5,2.0,2.3,3.1,3.2,0.2,0.9,1.8
305,Paul Reed,PHI,C,81,210,22,36.0,10.6,9.2,4.6,9.4,0.0,1.0,0.0,0.0,9.7,2.0,2.3,2.3,1.5
301,Matisse Thybulle,PHI,SG,77,201,24,36.0,7.9,8.2,3.3,7.1,1.1,3.7,0.4,0.5,3.2,1.5,1.7,2.6,1.4
304,Isaiah Joe,PHI,SG,76,165,22,36.0,6.1,10.6,3.4,10.6,2.5,7.7,1.3,1.5,3.9,2.3,0.2,0.7,1.1


In [19]:
results = {}
results = {team:[] for team in teams}
day = datetime.date(2021,10,19)
while day != datetime.date.today():
    for team in teams:
        try:
            for MU in games[str(day)]:
                if team in MU:
                    game = games[str(day)][MU][["Tm","W/L","Spread","Court","Opp"]]
                    game = [(game["W/L"][i],game["Spread"][i],game["Court"][i],game["Opp"][i]) for i,v in enumerate(game["Tm"]) if v==team][0]
                    results[team].append((day,game[0],game[1],game[2],game[3]))
        except:
            pass
    day = day+datetime.timedelta(days=1)
for team in teams:
    for result in results[team]:
        results[team] = pd.DataFrame(results[team],columns=["Date","W/L","Spread","Court","Opp"])
        results[team]["Rest"] = [(results[team]["Date"][i]-results[team]["Date"][i-1]-datetime.timedelta(1)).days if i!=0 else 3 for i in results[team]["Date"].index]


In [20]:
### TODO: Combine all team result dataframes together and find home court advantage, W% by days rest, etc.
# results["CHI"]["W"] = [True if i=="W" else False for i in results["CHI"]["W/L"]]
# results["CHI"]["L"] = [True if i=="L" else False for i in results["CHI"]["W/L"]]
# results["CHI"]["Home"] = [True if i=="Home" else False for i in results["CHI"]["Court"]]
# results["CHI"].aggregate({"W":"sum","L":"sum","Spread":"sum"})

What makes a good rebounder?
1) Height
2) Wingspan
3) Relative Weight
3) Positioning on the court when shots are taken
4) Hustle

How to check for good rebounders?
 - Compare each player's TRB to other players of the same size / wingspan
    - TRB-Ht: R = .459
    - TRB-Wt: R = .504
 - Check for a negative correlation for 3PA and TRB with centers (R = -.327)

In [21]:
# centers_per_36 = get_player_offense(filter="per_36",position="C")
# centers_per_36["PPI"] = round(centers_per_36["Wt"] / centers_per_36["Ht"],2)
# centers_per_36["zHT"] = round((centers_per_36["Ht"]-centers_per_36["Ht"].mean())/centers_per_36["Ht"].std(),2)
# centers_per_36["zPPI"]= round((centers_per_36["PPI"]-centers_per_36["PPI"].mean())/centers_per_36["PPI"].std(),2)
# centers_per_36["z3PA"]= -round((centers_per_36["3PA"]-centers_per_36["3PA"].mean())/centers_per_36["3PA"].std(),2)
# centers_per_36["Z"]= round(sum([centers_per_36["zHT"],centers_per_36["zPPI"],centers_per_36["z3PA"]])/3,2)

# centers_per_36.sort_values(by="Z",ascending=True)[:30].drop(columns=["Pos","MP","Wt","3P","3PA"])

### Compare two teams
# centers_per_36[(centers_per_36["Team"]=="MIA") | (centers_per_36["Team"]=="CHI")].drop(columns=["Pos","MP","Wt","3P","3PA"])

### Find stats that relate to each other
# centers_per_36.corr()

### Reveal normal rebounding range for position
# print(f'{centers_per_36["TRB"].mean()-centers_per_36["TRB"].std()} - {centers_per_36["TRB"].mean()+centers_per_36["TRB"].std()}')